In [1]:
!pip install pandas numpy mysql-connector-python sqlalchemy matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 34.1 MB/s eta 0:00:00


In [2]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os, shutil, sys\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import mysql.connector\n",
    "from sqlalchemy import create_engine\n",
    "import matplotlib.pyplot as plt\n",
    "import gzip\n",
    "#####\n",
    "SAMPLE_SIZE = 100\n",
    "new_names, new_model_names = [], []\n",
    "sample_sessions = []\n",
    "PATH_C3D_START = r\"Y:\\departments\\research_and_development\\sports_science\\01_mocap_operations\\pitching\\v6\\03_athletes\"\n",
    "PATH_C3D_COPY = r\"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\c3d\"\n",
    "PATH_META_SAVE = r\"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\"\n",
    "PATH_POI_SAVE = r\"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\poi\"\n",
    "PATH_FULL_SIG_SAVE = r\"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\full_sig\"\n",
    "SAVE_META_CSV = True\n",
    "# ###################### Connect to DB ######################\n",
    "db_user_ = os.environ['CLUSTER_USERNAME_DB_BIOMECH']\n",
    "db_pw_ = os.environ['CLUSTER_PASSWORD_DB_BIOMECH']\n",
    "db_dbname_ = os.environ['DATABASE_BIOMECH_PITCHING_DB']\n",
    "db_host_ = os.environ['CLUSTER_HOST_DB_BIOMECH']\n",
    "db_port_ = os.environ['CLUSTER_PORT_DB_BIOMECH']\n",
    "engine_string = 'mysql+mysqlconnector://'+db_user_ +':'+db_pw_+'@'+db_host_+':'+str(db_port_)+'/'+db_dbname_\n",
    "engine = create_engine(engine_string,connect_args={'auth_plugin': 'mysql_native_password'}, echo=False)\n",
    "\n",
    "#Get Data from Biomech DB\n",
    "cnx = mysql.connector.connect(host=db_host_,\n",
    "                                         database=db_dbname_,\n",
    "                                         user=db_user_,\n",
    "                                         password=db_pw_,\n",
    "                                         port =db_port_,\n",
    "                                         auth_plugin='mysql_native_password')\n",
    "\n",
    "cursor = cnx.cursor()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = 'select distinct session_pitch from bp_force_plate where rear_force_x is not null;'\n",
    "d = pd.read_sql(query, cnx)\n",
    "force_plate_pitches = tuple(pd.unique(d.session_pitch))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select u.user, s.session, poi.session_pitch, s.session_mass_kg, s.session_height_m, poi.pitch_speed_mph from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.irb=1) and (poi.pitch_type='FF') and (s.playing_level not in ('mlb', 'other')) and (poi.session_pitch in {}) and (u.user not in (552));\"\n",
    "d = pd.read_sql(query.format(force_plate_pitches), cnx)\n",
    "users = pd.unique(d.user).tolist()\n",
    "\n",
    "sample_users = np.random.choice(users, SAMPLE_SIZE, replace=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "for user in sample_users:\n",
    "    user_sessions = d[d.user==user].session.tolist()\n",
    "    # pick one session\n",
    "    session = np.random.choice(user_sessions, 1)[0]\n",
    "    sample_sessions.append(session)\n",
    "sample_sessions = tuple(sample_sessions)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select s.*, poi.session_pitch, poi.pitch_speed_mph, u.date_of_birth from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.session in {}) and (poi.pitch_type='FF');\"\n",
    "d1 = pd.read_sql(query.format(sample_sessions), cnx)\n",
    "d1['session_date'] = pd.to_datetime(d1['session_date'])\n",
    "d1['date_of_birth'] = pd.to_datetime(d1['date_of_birth'])\n",
    "d1['age_yrs'] = d1['session_date'] - d1['date_of_birth']\n",
    "d1['age_yrs'] = np.round(d1['age_yrs'].apply(lambda x: x.days/365.25),2)\n",
    "d1 = d1[['user', 'session', 'session_pitch', 'session_mass_kg', 'session_height_m', 'age_yrs', 'playing_level', 'pitch_speed_mph']]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select u.user, s.session, pos.session_pitch, pos.filename FROM raw_bp_positions pos left join bp_poi_metrics poi using(session_pitch) left join bp_sessions s on(s.SESSION=pos.session) left join bp_users u ON (s.user=u.user) where (s.session in {}) and (poi.pitch_type='FF') and u.name not in ('Luisa Gauci');\"\n",
    "d2 = pd.read_sql(query.format(sample_sessions), cnx)\n",
    "d2['user'] = d2['user'].astype(str).apply(lambda x: x.zfill(6))\n",
    "d2['session'] = d2['session'].astype(str).apply(lambda x: x.zfill(6))\n",
    "modelnames = d2['filename'].apply(lambda x: '_'.join(x.split('_')[0:3]) + '_model.c3d')\n",
    "d2['modelnames'] = modelnames\n",
    "\n",
    "for x in d2.iterrows():\n",
    "    parts = x[1]['filename'].split('_')\n",
    "    base_new = '_'.join(parts[3:])\n",
    "    base_new = x[1]['user'] + '_' + x[1]['session'] + '_' + base_new\n",
    "    base_new = base_new.replace('.json', '.c3d')\n",
    "    new_names.append(base_new)\n",
    "    model_new = x[1]['user'] + '_' + x[1]['session'] + '_model.c3d'\n",
    "    new_model_names.append(model_new)\n",
    "\n",
    "d2['filename_new'] = new_names\n",
    "d2['modelname_new'] = new_model_names"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# join d1 and d2 on session_pitch\n",
    "d = d1.merge(d2, on=['session_pitch'], how='left')\n",
    "d.drop(columns=['session_y', 'user_y'], inplace=True)\n",
    "d.rename(columns={'session_x': 'session', 'user_x': 'user'}, inplace=True)\n",
    "d.filename = d.filename.apply(lambda x: x.replace('.json', '.c3d'))\n",
    "sample_session_pitches = tuple(d.session_pitch.tolist())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if SAVE_META_CSV:\n",
    "    d.drop(columns=['filename', 'modelnames']).to_csv(os.path.join(PATH_META_SAVE, \"metadata.csv\"), index=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "for root, dirs, files in os.walk(PATH_C3D_START):\n",
    "    for file in files:\n",
    "        if file.endswith('.c3d'):\n",
    "            if file in d.filename.tolist():\n",
    "                filename_new = d[d.filename==file].filename_new.tolist()[0]\n",
    "                start_path = os.path.join(root, file)\n",
    "                folder_athlete = filename_new.split('_')[0]\n",
    "                end_path = os.path.join(PATH_C3D_COPY, folder_athlete, filename_new)\n",
    "                if not os.path.exists(os.path.join(PATH_C3D_COPY, folder_athlete)):\n",
    "                    os.makedirs(os.path.join(PATH_C3D_COPY, folder_athlete))\n",
    "                shutil.copyfile(start_path, end_path)\n",
    "                print('Copied {} to {}'.format(file, end_path))\n",
    "            elif file in d.modelnames.tolist():\n",
    "                filename_new = d[d.modelnames==file].modelname_new.tolist()[0]\n",
    "                start_path = os.path.join(root, file)\n",
    "                folder_athlete = filename_new.split('_')[0]\n",
    "                end_path = os.path.join(PATH_C3D_COPY, folder_athlete, filename_new)\n",
    "                if not os.path.exists(os.path.join(PATH_C3D_COPY, folder_athlete)):\n",
    "                    os.makedirs(os.path.join(PATH_C3D_COPY, folder_athlete))\n",
    "                shutil.copyfile(start_path, end_path)\n",
    "                print('Copied {} to {}'.format(file, end_path))\n",
    "            else:\n",
    "                pass"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# plt.rcParams.update({'font.family': 'gotham'})\n",
    "# fig = plt.figure(figsize=(10,12.5), facecolor='w')\n",
    "# # add subplots\n",
    "# ax0 = fig.add_subplot(311)\n",
    "# ax1 = fig.add_subplot(312)\n",
    "# ax2 = fig.add_subplot(313)\n",
    "\n",
    "# # plot data\n",
    "# ax0.hist(d.mass_lbs)\n",
    "# ax0.set_xlabel('Mass (lbs)')\n",
    "# ax0.set_ylabel('Count')\n",
    "\n",
    "# ax1.hist(d.height_in)\n",
    "# ax1.set_xlabel('Height (in)')\n",
    "# ax1.set_ylabel('Count')\n",
    "\n",
    "# ax2.hist(d.velo)\n",
    "# ax2.set_xlabel('Pitch Speed (mph)')\n",
    "# ax2.set_ylabel('Count')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "d = pd.read_csv(os.path.join(PATH_META_SAVE, 'metadata.csv'))\n",
    "sample_session_pitches = tuple(d.session_pitch.tolist())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# save force plate full signal data from session_pitches\n",
    "query = \"select fp.session_pitch, fp.time, fp.rear_force_x, fp.rear_force_y, fp.rear_force_z, fp.lead_force_x, fp.lead_force_y, fp.lead_force_z, e.pkh_time, e.fp_force_plates_time_10perc as fp_10_time, e.fp_force_plates_time_100perc as fp_100_time, e.MER_time, e.BR_time, e.MIR_time from bp_force_plate fp left join bp_events e using(session_pitch) where session_pitch in {};\".format(sample_session_pitches)\n",
    "force_plate_data = pd.read_sql(query, cnx)\n",
    "force_plate_data.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'force_plate.csv'), index=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select ef.*, e.pkh_time, e.fp_force_plates_time_10perc as fp_10_time, e.fp_force_plates_time_100perc as fp_100_time, e.MER_time, e.BR_time, e.MIR_time from bp_energy_flow ef left join bp_events e using(session_pitch) where session_pitch in {};\".format(sample_session_pitches)\n",
    "energy_flow_data = pd.read_sql(query, cnx)\n",
    "energy_flow_data.drop(columns=['session_pitch_time'], inplace=True)\n",
    "energy_flow_data.drop(columns=[x for x in energy_flow_data.columns if 'total' in x], inplace=True)\n",
    "energy_flow_data.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'energy_flow.csv'), index=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"SELECT jm.*, jf.*, e.pkh_time, e.fp_force_plates_time_10perc as fp_10_time, e.fp_force_plates_time_100perc as fp_100_time, e.MER_time, e.BR_time, e.MIR_time FROM bp_joint_forces jf LEFT JOIN bp_joint_moments jm USING(session_pitch_time) LEFT JOIN bp_events e ON(jm.session_pitch=e.session_pitch) WHERE jm.session_pitch in {};\".format(sample_session_pitches)\n",
    "forces_moments = pd.read_sql(query, cnx)\n",
    "forces_moments.drop(columns=['session_pitch_time'], inplace=True)\n",
    "forces_moments.iloc[:, 50:52] = np.nan\n",
    "forces_moments.dropna(axis=1, how='all', inplace=True)\n",
    "forces_moments.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'forces_moments.csv'), index=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select ja.*, e.pkh_time, e.fp_force_plates_time_10perc as fp_10_time, e.fp_force_plates_time_100perc as fp_100_time, e.MER_time, e.BR_time, e.MIR_time from bp_joint_angles ja left join bp_events e using(session_pitch) where session_pitch in {};\".format(sample_session_pitches)\n",
    "joint_angles = pd.read_sql(query, cnx)\n",
    "joint_angles.drop(columns=['session_pitch_time'], inplace=True)\n",
    "joint_angles.drop(columns=[x for x in joint_angles.columns if 'virtual_lab' in x], inplace=True)\n",
    "joint_angles.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'joint_angles.csv'), index=False)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select jv.*, e.pkh_time, e.fp_force_plates_time_10perc as fp_10_time, e.fp_force_plates_time_100perc as fp_100_time, e.MER_time, e.BR_time, e.MIR_time from bp_joint_velos jv left join bp_events e using(session_pitch) where session_pitch in {};\".format(sample_session_pitches)\n",
    "joint_velos = pd.read_sql(query, cnx)\n",
    "joint_velos.drop(columns=['session_pitch_time'], inplace=True)\n",
    "joint_velos.drop(columns=[x for x in joint_velos.columns if 'sig_mag' in x], inplace=True)\n",
    "joint_velos.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'joint_velos.csv'), index=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select lm.*, e.pkh_time, e.fp_force_plates_time_10perc as fp_10_time, e.fp_force_plates_time_100perc as fp_100_time, e.MER_time, e.BR_time, e.MIR_time from bp_landmarks lm left join bp_events e using(session_pitch) where session_pitch in {};\".format(sample_session_pitches)\n",
    "landmarks = pd.read_sql(query, cnx)\n",
    "landmarks.drop(columns=['session_pitch_time'], inplace=True)\n",
    "landmarks.drop(columns=[x for x in landmarks.columns if '_velo_' in x], inplace=True)\n",
    "landmarks.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'landmarks.csv'), index=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select * from bp_poi_metrics where session_pitch in {};\".format(sample_session_pitches)\n",
    "poi_metrics = pd.read_sql(query, cnx)\n",
    "poi_metrics.drop(columns=['lead_knee_extension_angular_velo_max_legacy'], inplace=True)\n",
    "poi_metrics.to_csv(os.path.join(PATH_POI_SAVE, 'poi_metrics.csv'), index=False)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3.8.5 ('v6_pitching_db')",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  },
  "orig_nbformat": 4,
  "vscode": {
   "interpreter": {
    "hash": "9d8a1536e469015d6bcd2696b378bfb01af892c1c6c53cf17a19284ec97cdbf9"
   }
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}

NameError: name 'null' is not defined

In [3]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os, shutil, sys\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import mysql.connector\n",
    "from sqlalchemy import create_engine\n",
    "import matplotlib.pyplot as plt\n",
    "import gzip\n",
    "#####\n",
    "SAMPLE_SIZE = 100\n",
    "new_names, new_model_names = [], []\n",
    "sample_sessions = []\n",
    "PATH_C3D_START = r\"Y:\\departments\\research_and_development\\sports_science\\01_mocap_operations\\pitching\\v6\\03_athletes\"\n",
    "PATH_C3D_COPY = r\"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\c3d\"\n",
    "PATH_META_SAVE = r\"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\"\n",
    "PATH_POI_SAVE = r\"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\poi\"\n",
    "PATH_FULL_SIG_SAVE = r\"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\full_sig\"\n",
    "SAVE_META_CSV = True\n",
    "# ###################### Connect to DB ######################\n",
    "db_user_ = os.environ['CLUSTER_USERNAME_DB_BIOMECH']\n",
    "db_pw_ = os.environ['CLUSTER_PASSWORD_DB_BIOMECH']\n",
    "db_dbname_ = os.environ['DATABASE_BIOMECH_PITCHING_DB']\n",
    "db_host_ = os.environ['CLUSTER_HOST_DB_BIOMECH']\n",
    "db_port_ = os.environ['CLUSTER_PORT_DB_BIOMECH']\n",
    "engine_string = 'mysql+mysqlconnector://'+db_user_ +':'+db_pw_+'@'+db_host_+':'+str(db_port_)+'/'+db_dbname_\n",
    "engine = create_engine(engine_string,connect_args={'auth_plugin': 'mysql_native_password'}, echo=False)\n",
    "\n",
    "#Get Data from Biomech DB\n",
    "cnx = mysql.connector.connect(host=db_host_,\n",
    "                                         database=db_dbname_,\n",
    "                                         user=db_user_,\n",
    "                                         password=db_pw_,\n",
    "                                         port =db_port_,\n",
    "                                         auth_plugin='mysql_native_password')\n",
    "\n",
    "cursor = cnx.cursor()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = 'select distinct session_pitch from bp_force_plate where rear_force_x is not null;'\n",
    "d = pd.read_sql(query, cnx)\n",
    "force_plate_pitches = tuple(pd.unique(d.session_pitch))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select u.user, s.session, poi.session_pitch, s.session_mass_kg, s.session_height_m, poi.pitch_speed_mph from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.irb=1) and (poi.pitch_type='FF') and (s.playing_level not in ('mlb', 'other')) and (poi.session_pitch in {}) and (u.user not in (552));\"\n",
    "d = pd.read_sql(query.format(force_plate_pitches), cnx)\n",
    "users = pd.unique(d.user).tolist()\n",
    "\n",
    "sample_users = np.random.choice(users, SAMPLE_SIZE, replace=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "for user in sample_users:\n",
    "    user_sessions = d[d.user==user].session.tolist()\n",
    "    # pick one session\n",
    "    session = np.random.choice(user_sessions, 1)[0]\n",
    "    sample_sessions.append(session)\n",
    "sample_sessions = tuple(sample_sessions)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "query = \"select s.*, poi.session_pitch, poi.pitch_speed_mph, u.date_of_birth from bp_sessions s left join bp_poi


SyntaxError: unterminated string literal (detected at line 90) (<ipython-input-3-9845a8a63815>, line 90)

In [4]:
import os
import shutil
import sys
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine

##### Define paths and parameters #####
SAMPLE_SIZE = 100
new_names, new_model_names = [], []
sample_sessions = []

PATH_C3D_START = r"Y:\departments\research_and_development\sports_science\01_mocap_operations\pitching\v6\03_athletes"
PATH_C3D_COPY = r"C:\Users\kylew\Documents\GitHub\openbiomechanics\baseball_pitching\data\c3d"
PATH_META_SAVE = r"C:\Users\kylew\Documents\GitHub\openbiomechanics\baseball_pitching\data"
PATH_POI_SAVE = r"C:\Users\kylew\Documents\GitHub\openbiomechanics\baseball_pitching\data\poi"
PATH_FULL_SIG_SAVE = r"C:\Users\kylew\Documents\GitHub\openbiomechanics\baseball_pitching\data\full_sig"
SAVE_META_CSV = True

##### Connect to the database #####
db_user_ = os.environ['CLUSTER_USERNAME_DB_BIOMECH']
db_pw_ = os.environ['CLUSTER_PASSWORD_DB_BIOMECH']
db_dbname_ = os.environ['DATABASE_BIOMECH_PITCHING_DB']
db_host_ = os.environ['CLUSTER_HOST_DB_BIOMECH']
db_port_ = os.environ['CLUSTER_PORT_DB_BIOMECH']

engine_string = 'mysql+mysqlconnector://' + db_user_ + ':' + db_pw_ + '@' + db_host_ + ':' + str(db_port_) + '/' + db_dbname_
engine = create_engine(engine_string, connect_args={'auth_plugin': 'mysql_native_password'}, echo=False)

# Get Data from Biomech DB
cnx = mysql.connector.connect(host=db_host_,
                               database=db_dbname_,
                               user=db_user_,
                               password=db_pw_,
                               port=db_port_,
                               auth_plugin='mysql_native_password')
cursor = cnx.cursor()

##### Query to get session pitches #####
query = 'select distinct session_pitch from bp_force_plate where rear_force_x is not null;'
d = pd.read_sql(query, cnx)
force_plate_pitches = tuple(pd.unique(d.session_pitch))

##### Query to get sample users #####
query = "select u.user, s.session, poi.session_pitch, s.session_mass_kg, s.session_height_m, poi.pitch_speed_mph from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.irb=1) and (poi.pitch_type='FF') and (s.playing_level not in ('mlb', 'other')) and (poi.session_pitch in {}) and (u.user not in (552));"
d = pd.read_sql(query.format(force_plate_pitches), cnx)
users = pd.unique(d.user).tolist()
sample_users = np.random.choice(users, SAMPLE_SIZE, replace=False)

##### Select random session pitches for sample users #####
for user in sample_users:
    user_sessions = d[d.user == user].session.tolist()
    session = np.random.choice(user_sessions, 1)[0]
    sample_sessions.append(session)
sample_sessions = tuple(sample_sessions)

##### Query to get metadata for sample session pitches #####
query = "select s.*, poi.session_pitch, poi.pitch_speed_mph, u.date_of_birth from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.session in {}) and (poi.pitch_type='FF');"
d1 = pd.read_sql(query.format(sample_sessions), cnx)
d1['session_date'] = pd.to_datetime(d1['session_date'])
d1['date_of_birth'] = pd.to_datetime(d1['date_of_birth'])
d1['age_yrs'] = d1['session_date'] - d1['date_of_birth']
d1['age_yrs'] = np.round(d1['age_yrs'].apply(lambda x: x.days / 365.25), 2)
d1 = d1[['user', 'session', 'session_pitch', 'session_mass_kg', 'session_height_m', 'age_yrs', 'playing_level', 'pitch_speed_mph']]

##### Query to get filenames for sample session pitches #####
query = "select u.user, s.session, pos.session_pitch, pos.filename FROM raw_bp_positions pos left join bp_poi_metrics poi using(session_pitch) left join bp_sessions s on(s.SESSION=pos.session) left join bp_users u ON (s.user=u.user) where (s.session in {}) and (poi.pitch_type='FF') and u.name not in ('Luisa Gauci');"
d2 = pd.read_sql(query.format(sample_sessions), cnx)
d2['user'] = d2['user'].astype(str).apply(lambda x: x.zfill(6


SyntaxError: incomplete input (<ipython-input-4-d2c1624fa206>, line 70)

In [5]:
import os
import shutil
import sys
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine

##### Define paths and parameters #####
SAMPLE_SIZE = 100
new_names, new_model_names = [], []
sample_sessions = []

PATH_C3D_START = "Y:/departments/research_and_development/sports_science/01_mocap_operations/pitching/v6/03_athletes"
PATH_C3D_COPY = "/content/c3d"
PATH_META_SAVE = "/content"
PATH_POI_SAVE = "/content/poi"
PATH_FULL_SIG_SAVE = "/content/full_sig"
SAVE_META_CSV = True

##### Connect to the database #####
db_user_ = os.environ['CLUSTER_USERNAME_DB_BIOMECH']
db_pw_ = os.environ['CLUSTER_PASSWORD_DB_BIOMECH']
db_dbname_ = os.environ['DATABASE_BIOMECH_PITCHING_DB']
db_host_ = os.environ['CLUSTER_HOST_DB_BIOMECH']
db_port_ = os.environ['CLUSTER_PORT_DB_BIOMECH']

engine_string = 'mysql+mysqlconnector://' + db_user_ + ':' + db_pw_ + '@' + db_host_ + ':' + str(db_port_) + '/' + db_dbname_
engine = create_engine(engine_string, connect_args={'auth_plugin': 'mysql_native_password'}, echo=False)

# Get Data from Biomech DB
cnx = mysql.connector.connect(host=db_host_,
                               database=db_dbname_,
                               user=db_user_,
                               password=db_pw_,
                               port=db_port_,
                               auth_plugin='mysql_native_password')
cursor = cnx.cursor()

##### Query to get session pitches #####
query = 'select distinct session_pitch from bp_force_plate where rear_force_x is not null;'
d = pd.read_sql(query, cnx)
force_plate_pitches = tuple(pd.unique(d.session_pitch))

##### Query to get sample users #####
query = "select u.user, s.session, poi.session_pitch, s.session_mass_kg, s.session_height_m, poi.pitch_speed_mph from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.irb=1) and (poi.pitch_type='FF') and (s.playing_level not in ('mlb', 'other')) and (poi.session_pitch in {}) and (u.user not in (552));"
d = pd.read_sql(query.format(force_plate_pitches), cnx)
users = pd.unique(d.user).tolist()
sample_users = np.random.choice(users, SAMPLE_SIZE, replace=False)

##### Select random session pitches for sample users #####
for user in sample_users:
    user_sessions = d[d.user == user].session.tolist()
    session = np.random.choice(user_sessions, 1)[0]
    sample_sessions.append(session)
sample_sessions = tuple(sample_sessions)

##### Query to get metadata for sample session pitches #####
query = "select s.*, poi.session_pitch, poi.pitch_speed_mph, u.date_of_birth from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.session in {}) and (poi.pitch_type='FF');"
d1 = pd.read_sql(query.format(sample_sessions), cnx)
d1['session_date'] = pd.to_datetime(d1['session_date'])
d1['date_of_birth'] = pd.to_datetime(d1['date_of_birth'])
d1['age_yrs'] = d1['session_date'] - d1['date_of_birth']
d1['age_yrs'] = np.round(d1['age_yrs'].apply(lambda x: x.days / 365.25), 2)
d1 = d1[['user', 'session', 'session_pitch', 'session_mass_kg', 'session_height_m', 'age_yrs', 'playing_level', 'pitch_speed_mph']]

##### Query to get filenames for sample session pitches #####
query = "select u.user, s.session, pos.session_pitch, pos.filename FROM raw_bp_positions pos left join bp_poi_metrics poi using(session_pitch) left join bp_sessions s on(s.SESSION=pos.session) left join bp_users u ON (s.user=u.user) where (s.session in {}) and (poi.pitch_type='FF') and u.name not in ('Luisa Gauci');"
d2 = pd.read_sql(query.format(sample_sessions), cnx)
d2['user'] = d2['user'].astype(str).apply(lambda x: x.zfill(6))


KeyError: 'CLUSTER_USERNAME_DB_BIOMECH'

In [6]:
import os, shutil, sys
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import gzip

##### Definir variables de entorno manualmente #####
os.environ['CLUSTER_USERNAME_DB_BIOMECH'] = 'mi_usuario'
os.environ['CLUSTER_PASSWORD_DB_BIOMECH'] = 'mi_contraseña'
os.environ['DATABASE_BIOMECH_PITCHING_DB'] = 'mi_basededatos'
os.environ['CLUSTER_HOST_DB_BIOMECH'] = 'mi_host'
os.environ['CLUSTER_PORT_DB_BIOMECH'] = 'mi_puerto'

##### Otras configuraciones #####
SAMPLE_SIZE = 100
new_names, new_model_names = [], []
sample_sessions = []
PATH_C3D_START = r"Y:\\departments\\research_and_development\\sports_science\\01_mocap_operations\\pitching\\v6\\03_athletes"
PATH_C3D_COPY = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\c3d"
PATH_META_SAVE = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data"
PATH_POI_SAVE = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\poi"
PATH_FULL_SIG_SAVE = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\full_sig"
SAVE_META_CSV = True

##### Connect to the database #####
db_user_ = os.environ['CLUSTER_USERNAME_DB_BIOMECH']
db_pw_ = os.environ['CLUSTER_PASSWORD_DB_BIOMECH']
db_dbname_ = os.environ['DATABASE_BIOMECH_PITCHING_DB']
db_host_ = os.environ['CLUSTER_HOST_DB_BIOMECH']
db_port_ = os.environ['CLUSTER_PORT_DB_BIOMECH']
engine_string = 'mysql+mysqlconnector://'+db_user_ +':'+db_pw_+'@'+db_host_+':'+str(db_port_)+'/'+db_dbname_
engine = create_engine(engine_string,connect_args={'auth_plugin': 'mysql_native_password'}, echo=False)

#Get Data from Biomech DB
cnx = mysql.connector.connect(host=db_host_,
                                       database=db_dbname_,
                                       user=db_user_,
                                       password=db_pw_,
                                       port =db_port_,
                                       auth_plugin='mysql_native_password')

cursor = cnx.cursor()

query = 'select distinct session_pitch from bp_force_plate where rear_force_x is not null;'
d = pd.read_sql(query, cnx)
force_plate_pitches = tuple(pd.unique(d.session_pitch))

query = "select u.user, s.session, poi.session_pitch, s.session_mass_kg, s.session_height_m, poi.pitch_speed_mph from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.irb=1) and (poi.pitch_type='FF') and (s.playing_level not in ('mlb', 'other')) and (poi.session_pitch in {}) and (u.user not in (552));"
d = pd.read_sql(query.format(force_plate_pitches), cnx)
users = pd.unique(d.user).tolist()

sample_users = np.random.choice(users, SAMPLE_SIZE, replace=False)

for user in sample_users:
    user_sessions = d[d.user==user].session.tolist()
    # pick one session
    session = np.random.choice(user_sessions, 1)[0]
    sample_sessions.append(session)
sample_sessions = tuple(sample_sessions)

query = "select s.*, poi.session_pitch, poi.pitch_speed_mph, u.date_of_birth from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.session in {}) and (poi.pitch_type='FF');"
d1 = pd.read_sql(query.format(sample_sessions), cnx)
d1['session_date'] = pd.to_datetime(d1['session_date'])
d1['date_of_birth'] = pd.to_datetime(d1['date_of_birth'])
d1['age_yrs'] = d1['session_date'] - d1['date_of_birth']
d1['age_yrs'] = np.round(d1['age_yrs'].apply(lambda x: x.days/365.25),2)
d1 = d1[['user', 'session', 'session_pitch', 'session_mass_kg', 'session_height_m', 'age_yrs', 'playing_level', 'pitch_speed_mph']]

query = "select u.user, s.session, pos.session_pitch, pos.filename FROM raw_bp_positions pos left join bp_poi_metrics poi using(session_pitch) left join bp_sessions s on(s.SESSION=pos.session) left join bp_users u ON (s.user=u.user) where (s.session in {}) and (poi.pitch_type='FF') and u.name not in ('Luisa Gauci');"
d2 = pd.read_sql(query.format(sample_sessions), cnx)
d2['user'] = d2['user'].astype(str).apply(lambda x: x.zfill(6))
d2['session'] = d2['session'].astype(str).apply(lambda x: x.zfill(6))
modelnames = d2['filename'].apply(lambda x: '_


SyntaxError: unterminated string literal (detected at line 75) (<ipython-input-6-ad1781244d84>, line 75)

In [7]:
import os
import shutil
import sys
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import gzip

##### Constants #####
SAMPLE_SIZE = 100
new_names, new_model_names = [], []
sample_sessions = []
PATH_C3D_START = r"Y:\\departments\\research_and_development\\sports_science\\01_mocap_operations\\pitching\\v6\\03_athletes"
PATH_C3D_COPY = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\c3d"
PATH_META_SAVE = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data"
PATH_POI_SAVE = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\poi"
PATH_FULL_SIG_SAVE = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\full_sig"
SAVE_META_CSV = True

##### Connect to the database #####
# Assign your own values to these variables
db_user_ = "your_username"
db_pw_ = "your_password"
db_dbname_ = "your_db_name"
db_host_ = "your_host"
db_port_ = "your_port"

engine_string = 'mysql+mysqlconnector://' + db_user_ + ':' + db_pw_ + '@' + db_host_ + ':' + str(
    db_port_) + '/' + db_dbname_
engine = create_engine(engine_string, connect_args={'auth_plugin': 'mysql_native_password'}, echo=False)

# Get Data from Biomech DB
cnx = mysql.connector.connect(host=db_host_,
                               database=db_dbname_,
                               user=db_user_,
                               password=db_pw_,
                               port=db_port_,
                               auth_plugin='mysql_native_password')

cursor = cnx.cursor()

query = 'select distinct session_pitch from bp_force_plate where rear_force_x is not null;'
d = pd.read_sql(query, cnx)
force_plate_pitches = tuple(pd.unique(d.session_pitch))

query = "select u.user, s.session, poi.session_pitch, s.session_mass_kg, s.session_height_m, poi.pitch_speed_mph from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.irb=1) and (poi.pitch_type='FF') and (s.playing_level not in ('mlb', 'other')) and (poi.session_pitch in {}) and (u.user not in (552));"
d = pd.read_sql(query.format(force_plate_pitches), cnx)
users = pd.unique(d.user).tolist()

sample_users = np.random.choice(users, SAMPLE_SIZE, replace=False)

for user in sample_users:
    user_sessions = d[d.user == user].session.tolist()
    # pick one session
    session = np.random.choice(user_sessions, 1)[0]
    sample_sessions.append(session)

sample_sessions = tuple(sample_sessions)

query = "select s.*, poi.session_pitch, poi.pitch_speed_mph, u.date_of_birth from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.session in {}) and (poi.pitch_type='FF');"
d1 = pd.read_sql(query.format(sample_sessions), cnx)
d1['session_date'] = pd.to_datetime(d1['session_date'])
d1['date_of_birth'] = pd.to_datetime(d1['date_of_birth'])
d1['age_yrs'] = d1['session_date'] - d1['date_of_birth']
d1['age_yrs'] = np.round(d1['age_yrs'].apply(lambda x: x.days / 365.25), 2)
d1 = d1[['user', 'session', 'session_pitch', 'session_mass_kg', 'session_height_m', 'age_yrs', 'playing_level',
         'pitch_speed_mph']]

query = "select u.user, s.session, pos.session_pitch, pos.filename FROM raw_bp_positions pos left join bp_poi_metrics poi using(session_pitch) left join bp_sessions s on(s.SESSION=pos.session) left join bp_users u ON (s.user=u.user) where (s.session in {}) and (poi.pitch_type='FF') and u.name not in ('Luisa Gauci');"
d2 = pd.read_sql(query.format(sample_sessions), cnx)
d2['user'] = d2['user'].astype(str).apply(lambda x: x.zfill(6))
d2['session'] = d2['session'].astype(str).apply(lambda x: x.zfill(6))
modelnames = d2['filename'].apply(lambda x: '_'.join(x.split('_')[0:3]) + '_model.c3d')
d2['model


SyntaxError: unterminated string literal (detected at line 76) (<ipython-input-7-a17da82be04f>, line 76)

In [9]:
import os
import shutil
import sys
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import gzip

##### Constants #####
SAMPLE_SIZE = 100
new_names, new_model_names = [], []
sample_sessions = []
PATH_C3D_START = r"Y:\\departments\\research_and_development\\sports_science\\01_mocap_operations\\pitching\\v6\\03_athletes"
PATH_C3D_COPY = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\c3d"
PATH_META_SAVE = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data"
PATH_POI_SAVE = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\poi"
PATH_FULL_SIG_SAVE = r"C:\\Users\\kylew\\Documents\\GitHub\\openbiomechanics\\baseball_pitching\\data\\full_sig"
SAVE_META_CSV = True

##### Connect to the database #####
# Assign your own values to these variables
db_user_ = "Samuel Hidalgo"
db_pw_ = "Italia1234"
db_dbname_ = "Samuel"
db_host_ = "Junior"
db_port_ = "Parma"

engine_string = 'mysql+mysqlconnector://' + db_user_ + ':' + db_pw_ + '@' + db_host_ + ':' + str(
    db_port_) + '/' + db_dbname_
engine = create_engine(engine_string, connect_args={'auth_plugin': 'mysql_native_password'}, echo=False)

# Get Data from Biomech DB
cnx = mysql.connector.connect(host=db_host_,
                               database=db_dbname_,
                               user=db_user_,
                               password=db_pw_,
                               port=db_port_,
                               auth_plugin='mysql_native_password')

cursor = cnx.cursor()

query = 'select distinct session_pitch from bp_force_plate where rear_force_x is not null;'
d = pd.read_sql(query, cnx)
force_plate_pitches = tuple(pd.unique(d.session_pitch))

query = "select u.user, s.session, poi.session_pitch, s.session_mass_kg, s.session_height_m, poi.pitch_speed_mph from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.irb=1) and (poi.pitch_type='FF') and (s.playing_level not in ('mlb', 'other')) and (poi.session_pitch in {}) and (u.user not in (552));"
d = pd.read_sql(query.format(force_plate_pitches), cnx)
users = pd.unique(d.user).tolist()

sample_users = np.random.choice(users, SAMPLE_SIZE, replace=False)

for user in sample_users:
    user_sessions = d[d.user == user].session.tolist()
    # pick one session
    session = np.random.choice(user_sessions, 1)[0]
    sample_sessions.append(session)

sample_sessions = tuple(sample_sessions)

query = "select s.*, poi.session_pitch, poi.pitch_speed_mph, u.date_of_birth from bp_sessions s left join bp_poi_metrics poi using(session) left join bp_users u using(user) where (s.session in {}) and (poi.pitch_type='FF');"
d1 = pd.read_sql(query.format(sample_sessions), cnx)
d1['session_date'] = pd.to_datetime(d1['session_date'])
d1['date_of_birth'] = pd.to_datetime(d1['date_of_birth'])
d1['age_yrs'] = d1['session_date'] - d1['date_of_birth']
d1['age_yrs'] = np.round(d1['age_yrs'].apply(lambda x: x.days / 365.25), 2)
d1 = d1[['user', 'session', 'session_pitch', 'session_mass_kg', 'session_height_m', 'age_yrs', 'playing_level',
         'pitch_speed_mph']]

query = "select u.user, s.session, pos.session_pitch, pos.filename FROM raw_bp_positions pos left join bp_poi_metrics poi using(session_pitch) left join bp_sessions s on(s.SESSION=pos.session) left join bp_users u ON (s.user=u.user) where (s.session in {}) and (poi.pitch_type='FF') and u.name not in ('Luisa Gauci');"
d2 = pd.read_sql(query.format(sample_sessions), cnx)
d2['user'] = d2['user'].astype(str).apply(lambda x: x.zfill(6))
d2['session'] = d2['session'].astype(str).apply(lambda x: x.zfill(6))
modelnames = d2['filename'].apply



ValueError: invalid literal for int() with base 10: 'Parma'

In [10]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="styles.css">
    <title>Contador de Pitcheos</title>
</head>
<body>
    <div class="container">
        <div class="counter">
            <p>Pitcheos: <span id="pitchCount">0</span></p>
            <p>Bolas: <span id="ballCount">0</span></p>
            <p>Strikes: <span id="strikeCount">0</span></p>
            <p>Porcentaje de Strikes: <span id="strikePercentage">0%</span></p>
            <label for="pitchType">Tipo de Pitcheo:</label>
            <select id="pitchType" onchange="selectPitchType()">
                <option value="Fastball">Fastball</option>
                <option value="Curveball">Curveball</option>
                <option value="Slider">Slider</option>
                <option value="Change up">Change up</option>
                <option value="Splitter">Splitter</option>
                <option value="Cutter">Cutter</option>
                <option value="Sinker">Sinker</option>
            </select>
            <button onclick="incrementBall()">Bola</button>
            <button onclick="incrementStrike()">Strike</button>
        </div>
    </div>
    <script src="script.js"></script>
</body>
</html>


SyntaxError: invalid syntax (<ipython-input-10-506f2bad8340>, line 1)

In [11]:
from ipywidgets import interact, widgets

pitchCount = 0
ballCount = 0
strikeCount = 0

def update_counts():
    strike_percentage = "{:.2f}%".format((strikeCount / pitchCount) * 100) if pitchCount > 0 else "0%"
    pitch_count_label.value = str(pitchCount)
    ball_count_label.value = str(ballCount)
    strike_count_label.value = str(strikeCount)
    strike_percentage_label.value = strike_percentage

def select_pitch_type(pitch_type):
    print("Tipo de Pitcheo:", pitch_type)

def increment_ball(b):
    global pitchCount, ballCount
    pitchCount += 1
    ballCount += 1
    update_counts()

def increment_strike(s):
    global pitchCount, strikeCount
    pitchCount += 1
    strikeCount += 1
    update_counts()

pitch_count_label = widgets.Label(value="0")
ball_count_label = widgets.Label(value="0")
strike_count_label = widgets.Label(value="0")
strike_percentage_label = widgets.Label(value="0%")

interact(select_pitch_type, pitch_type=widgets.Dropdown(options=["Fastball", "Curveball", "Slider", "Change up", "Splitter", "Cutter", "Sinker"]))
ball_button = widgets.Button(description="Bola")
strike_button = widgets.Button(description="Strike")

ball_button.on_click(increment_ball)
strike_button.on_click(increment_strike)

widgets.VBox([
    widgets.Label(value="Pitcheos:"),
    pitch_count_label,
    widgets.Label(value="Bolas:"),
    ball_count_label,
    widgets.Label(value="Strikes:"),
    strike_count_label,
    widgets.Label(value="Porcentaje de Strikes:"),
    strike_percentage_label,
    widgets.HBox([ball_button, strike_button])
])


interactive(children=(Dropdown(description='pitch_type', options=('Fastball', 'Curveball', 'Slider', 'Change u…